In [1]:
import sys
sys.path.append('../py_scripts')

In [2]:
import keras
import os
from model_utils import get_mobilenet_model
from utils import get_train_test_data_dict, DictImageDataGenerator, preprocess_func

# define train params
IMG_SIZE = 224
LABEL_CNT = 5005
ALL_DATA_JSON = '../data/train_data.json'
BATCH_SIZE = 8
print('load done')


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


load done


In [3]:
# get generator
train_d, val_d = get_train_test_data_dict(ALL_DATA_JSON, 0.1)
train_ds = DictImageDataGenerator(rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  preprocessing_function=preprocess_func)
train_gen = train_ds.flow_from_dict(train_d, target_size=(
    IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)
val_ds = DictImageDataGenerator(preprocessing_function=preprocess_func)
val_gen = val_ds.flow_from_dict(val_d, target_size=(
    IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)
val_steps = val_gen.samples//BATCH_SIZE
print(val_steps)
# test
for x, y in train_gen:
    print(x.shape, y.shape)
    break

Found 23996 images belonging to 5005 classes.
Found 12248 images belonging to 5005 classes.
1531
(8, 224, 224, 3) (8, 5005)


In [4]:
# get baseline mobilenet model
mob_model = get_mobilenet_model(IMG_SIZE, LABEL_CNT, dense_dim=256)
mob_model.summary()

/usr/local/lib/python3.5/dist-packages/keras_applications/mobilenet.py:206: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [5]:
# compile
def get_callbacks(model_save_path, model=None):
    if os.path.exists(model_save_path):
        if model is not None:
            model.load_weights(model_save_path)
    model_dir = os.path.dirname(model_save_path)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    m_reduce = keras.callbacks.ReduceLROnPlateau(
        monitor='loss', factor=0.5, patience=3, verbose=1, min_lr=0.00000001)
    m_check = keras.callbacks.ModelCheckpoint(
        model_save_path, monitor='val_loss', verbose=1, save_best_only=True)
    return [m_reduce, m_check]

cb_list = get_callbacks('../data/checkpoints/mob_base.h5', mob_model)
mob_model.compile(optimizer=adam_opt,
                  loss='categorical_crossentropy', metrics=['acc'])
print('compile done')

compile done


In [6]:
# train
mob_model.fit_generator(
    train_gen,
    steps_per_epoch=2000,
    epochs=100,
    verbose=1,
    callbacks=cb_list,
    validation_data=val_gen,
    validation_steps=val_steps
)

Epoch 1/100
2000/2000 [==============================] - 1944s 972ms/step - loss: 11.6472 - acc: 0.3596 - val_loss: 10.8742 - val_acc: 0.0789

Epoch 00001: val_loss improved from inf to 10.87423, saving model to ../data/checkpoints/mob_base.h5
Epoch 2/100
2000/2000 [==============================] - 1907s 954ms/step - loss: 10.9381 - acc: 0.3618 - val_loss: 9.2746 - val_acc: 0.0789

Epoch 00002: val_loss improved from 10.87423 to 9.27458, saving model to ../data/checkpoints/mob_base.h5
Epoch 3/100
2000/2000 [==============================] - 1899s 949ms/step - loss: 11.1265 - acc: 0.3656 - val_loss: 10.9416 - val_acc: 0.0789

Epoch 00003: val_loss did not improve from 9.27458
Epoch 4/100
2000/2000 [==============================] - 1906s 953ms/step - loss: 10.4271 - acc: 0.3623 - val_loss: 11.0415 - val_acc: 0.0789

Epoch 00004: val_loss did not improve from 9.27458
Epoch 5/100
2000/2000 [==============================] - 1907s 953ms/step - loss: 10.0223 - acc: 0.3587 - val_loss: 10.71


Epoch 00039: val_loss did not improve from 7.91288
Epoch 40/100
2000/2000 [==============================] - 2061s 1s/step - loss: 9.1380 - acc: 0.3619 - val_loss: 8.6407 - val_acc: 0.0789

Epoch 00040: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00040: val_loss did not improve from 7.91288
Epoch 41/100
 576/2000 [=======>......................] - ETA: 22:02 - loss: 9.2693 - acc: 0.3557

KeyboardInterrupt: 